# Probability lab simulations

Below are some simulations for the lab exercises. Sometimes, we can't easily find an answer to a probability  problem using theory. This notebook shows how we can find some answers using simulation instead.

First, we need to import some libraries.

In [1]:
import numpy as np  # Math
import random       # Randomness
import matplotlib.pylab as plt  # Plotting
import ipywidgets as widgets    # Interactive stuff

## Throwing a coin or a die

The following widget shows the come of throwin one (or more) coins or dice. The simulation assumes there is a set number of outcomes (```n_outcomes```) and that all outcomes are equally likely.

In [2]:
n_outcomes = 2

data = list()
outcomes = list(range(1, n_outcomes+1))
print("Possible outcomes:", outcomes)

def throw_dice(n_throws):
  # Throw the die/coin
  new_data = random.choices(outcomes, k=n_throws)
  data.extend(new_data)
  print("New data (n=%i): %s" % (len(new_data), new_data))
  if len(data) < 50:
    print("All datapoints:", data)
  else:
    print("Last data points: ...", data[-50:])
  # Define the figures
  fig = plt.figure(figsize=(15, 4), dpi=100)
  ax = fig.subplots(1, 2)
  ax[0].set_ylabel("# throws")
  ax[0].set_xlabel('Outcomes')
  ax[1].set_xlabel("# throws")
  ax[1].set_ylabel('Probability')
  # Plot data
  data_counts = np.zeros(n_outcomes)
  new_data_counts = np.zeros(n_outcomes)
  x = np.arange(1, len(data)+1)
  for i, outcome in enumerate(outcomes):
    data_counts[i] = np.sum(np.asarray(data)==outcome)
    new_data_counts[i] = np.sum(np.asarray(new_data)==outcome)
    ax[1].plot(x, np.cumsum(np.asarray(data)==outcome)/x, 
               alpha=.7, label="Prob. of outcome %i" % outcome)
  ax[1].plot(x, np.ones(x.shape)/n_outcomes, 'g--', alpha=.5, label="Theoretical prob.")
  if n_outcomes == 2:
    labels = ['Heads', 'Tails']
  else:
    labels = ["Side %i" % (i+1) for i in range(n_outcomes)]
  colours = ["C%i" % i for i in range(n_outcomes)]
  old_data_counts = data_counts-new_data_counts
  ax[0].bar(labels, new_data_counts, bottom=old_data_counts, align='center', color='red', alpha=.6)
  ax[0].bar(labels, old_data_counts, align='center', color=colours, alpha=.7)
  # Adjust graphs and commit to screen
  a = list(ax[0].axis())
  a[3] = max(a[3], 25)
  ax[0].axis(tuple(a))
  ax[1].legend(loc='upper right')
  fig.show()

widgets.interact_manual.opts['manual_name'] = 'Throw dice!'
widgets.interact_manual(throw_dice, n_throws=widgets.IntSlider(min=1, max=100, step=1, value=1));

Possible outcomes: [1, 2]


interactive(children=(IntSlider(value=1, description='n_throws', min=1), Button(description='Throw dice!', sty…

## Peach's and Yoshi's dice game

The first thing to do is to define a set of outcomes.

In [3]:
outcomes = list()
for die1 in range(1, 6+1):
  for die2 in range(1, 6+1):
    outcomes.append((die1, die2))
print("%i outcomes: %s" % (len(outcomes), outcomes))

36 outcomes: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)]


Let's pick a few outcomes and show the game outcome.

In [4]:
def peach_wins(outcome):
  die1, die2 = outcome
  diff = abs( die1 - die2 )
  if diff <= 2:
    return True
  else:
    return False

for outcome in random.choices(outcomes, k=5):
  print("Outcome drawn: %s" % repr(outcome))
  if peach_wins(outcome):
    print("  -> Peach wins!")
  else:
    print("  -> Yoshi wins!")
  print()

Outcome drawn: (5, 2)
  -> Yoshi wins!

Outcome drawn: (4, 3)
  -> Peach wins!

Outcome drawn: (1, 1)
  -> Peach wins!

Outcome drawn: (4, 2)
  -> Peach wins!

Outcome drawn: (4, 6)
  -> Peach wins!



In [5]:
data = list()

def throw_dice(n_throws):
  # Throw the dice
  new_data = random.choices(outcomes, k=n_throws)
  data.extend(new_data)
  print("New data (n=%i): %s" % (len(new_data), new_data))
  if len(data) < 50:
    print("All datapoints:", data)
  else:
    print("Last data points: ...", data[-50:])
  # Define the figures
  fig = plt.figure(figsize=(15, 4), dpi=100)
  ax = fig.subplots(1, 2)
  ax[0].set_ylabel("# games")
  n = np.sum([peach_wins(outcome) for outcome in data])
  m = np.sum([not peach_wins(outcome) for outcome in data])
  ax[0].set_xlabel("Diff. for games (Yoshi: %i, Peach: %i)" %(m, n))
  ax[1].set_xlabel("# games (total of %i)" % len(data))
  ax[1].set_ylabel('Probability')
  # Plot data
  diff_counts = np.zeros(6)
  for outcome in data:
    diff_counts[abs(outcome[0]-outcome[1])] += 1
  new_diff_counts = np.zeros(6)
  for outcome in new_data:
    new_diff_counts[abs(outcome[0]-outcome[1])] += 1
  games = np.arange(1, len(data)+1)
  ax[1].plot(games, np.cumsum([peach_wins(outcome) for outcome in data])/games, 
              color='pink', label="Prob. of Peach winning")
  ax[1].plot(games, np.cumsum([not peach_wins(outcome) for outcome in data])/games, 
              color='green', alpha=.7, label="Prob. of Yoshi winning")
  # ax[1].plot(x, np.ones(x.shape)/n_outcomes, 'g--', alpha=.5, label="Theoretical prob.")
  labels = ["%i" % i for i in range(6)]
  colours = ['pink', 'pink', 'pink', 'green', 'green', 'green']
  old_diff_counts = diff_counts-new_diff_counts
  ax[0].bar(labels, new_diff_counts, bottom=old_diff_counts, align='center', color='red', alpha=.5)
  ax[0].bar(labels, old_diff_counts, align='center', color=colours, alpha=.9)
  # Adjust graphs and commit to screen
  a = list(ax[0].axis())
  a[2] = 0
  a[3] = max(np.max(diff_counts)+10, 25)
  ax[0].axis(tuple(a))
  a = list(ax[1].axis())
  a[2] = 0
  a[3] = 1
  ax[1].axis(tuple(a))
  ax[1].legend(loc='upper right')
  fig.show()

widgets.interact_manual.opts['manual_name'] = 'Throw dice!'
widgets.interact_manual(throw_dice, n_throws=widgets.IntSlider(min=1, max=100, step=1, value=1));

interactive(children=(IntSlider(value=1, description='n_throws', min=1), Button(description='Throw dice!', sty…

## Monty Hall

A good first step is to define the set of outcomes.

In [6]:
outcomes = list()
for door in [1, 2, 3]:
  for choice in [1, 2, 3]:
    for change in [True, False]:
      outcomes.append((door, choice, change))
print("%i outcomes: %s " % (len(outcomes), outcomes))

18 outcomes: [(1, 1, True), (1, 1, False), (1, 2, True), (1, 2, False), (1, 3, True), (1, 3, False), (2, 1, True), (2, 1, False), (2, 2, True), (2, 2, False), (2, 3, True), (2, 3, False), (3, 1, True), (3, 1, False), (3, 2, True), (3, 2, False), (3, 3, True), (3, 3, False)] 


Next, we need to define a win contition. I've chosen to do this as a python function. It is good practice to always try out your code. Especially when it is so easy to do. To be extra clear *do not trust your code, always test it*.

In [7]:
def did_we_win(outcome):
  door, choice, change = outcome
  if door==choice and not change:
    return True
  elif door != choice and change:
    return True
  else:
    return False

for outcome in random.choices(outcomes, k=5):
  print("Outcome drawn: %s" % repr(outcome))
  door, choice, change = outcome
  print(" Door with car:", door)
  print(" We chose door:", choice)
  if change:
    print(" We changed our choice")
  else:
    print(" We stood by our choice")
  if did_we_win(outcome):
    print("  -> We won a car!")
  else:
    print("  -> We won a goat :(")
  print()

Outcome drawn: (2, 3, False)
 Door with car: 2
 We chose door: 3
 We stood by our choice
  -> We won a goat :(

Outcome drawn: (1, 1, True)
 Door with car: 1
 We chose door: 1
 We changed our choice
  -> We won a goat :(

Outcome drawn: (3, 3, True)
 Door with car: 3
 We chose door: 3
 We changed our choice
  -> We won a goat :(

Outcome drawn: (2, 1, True)
 Door with car: 2
 We chose door: 1
 We changed our choice
  -> We won a car!

Outcome drawn: (2, 2, False)
 Door with car: 2
 We chose door: 2
 We stood by our choice
  -> We won a car!



### A fast way of finding the probability.

We can assume all outcomes are equally likely and simply go through them.
 
**Theory:**
 
Conditional probability is defined as: $P(A|B) = \frac{P(A \cap B)}{P(B)}$. If we define winning as event A and changing door as event B, event B's compliment will be not changing door. Since each outcome is either in B or not, then $P(B^c)=1-P(B)$.

In [8]:
A_and_B = 0
A_and_Bc = 0
B = 0
for outcome in outcomes:
  door, choice, change = outcome
  if did_we_win(outcome): # Event A, i.e. winning
    if change:            # Event A and B
      A_and_B += 1
    else:                 # Event A and B^c, i.e. not change door
      A_and_Bc += 1
  if change:              # Event B
    B += 1

print("p(win | changed door)    = %.1f%%" % (100*A_and_B/B))
print("p(win | not change door) = %.1f%%" % (100*A_and_Bc/(len(outcomes)-B)))

p(win | changed door)    = 66.7%
p(win | not change door) = 33.3%


### A fun way of finding the probability 

Through simulation.

In [9]:
data = list()

def run_game_show(n_runs):
  # Run the show
  new_data = random.choices(outcomes, k=n_runs)
  data.extend(new_data)
  print("New data (n=%i): %s" % (len(new_data), new_data))
  if len(data) < 50:
    print("All datapoints:", data)
  else:
    print("Last data points: ...", data[-50:])
  # Define the figures
  fig = plt.figure(figsize=(15, 4), dpi=100)
  ax = fig.subplots(1, 2)
  ax[0].set_ylabel("# shows")
  losses = np.sum([not did_we_win(outcome) for outcome in data])
  ax[0].set_xlabel("Outcomes (%i losses not counted)" % losses)
  ax[1].set_xlabel("# shows")
  ax[1].set_ylabel('Probability')
  # Plot data

  strategy_counts = np.zeros(2)
  new_strategy_counts = np.zeros(2)
  games = np.arange(1, len(data)+1)
  labels = ['win | change', 'win | no change']
  colours = ["C0", "C1"]

  strategy_counts[0] = np.sum([did_we_win(outcome) and outcome[2] for outcome in data])
  strategy_counts[1] = np.sum([did_we_win(outcome) and not outcome[2] for outcome in data])
  new_strategy_counts[0] = np.sum([did_we_win(outcome) and outcome[2] for outcome in new_data])
  new_strategy_counts[1] = np.sum([did_we_win(outcome) and not outcome[2] for outcome in new_data])
  old_strategy_counts = strategy_counts-new_strategy_counts
  ax[0].bar(labels, new_strategy_counts, bottom=old_strategy_counts, align='center', color='red', alpha=.6)
  ax[0].bar(labels, old_strategy_counts, align='center', color=colours, alpha=.7)
  
  A_B = np.cumsum([did_we_win(outcome) and outcome[2] for outcome in data])
  A_Bc = np.cumsum([did_we_win(outcome) and not outcome[2] for outcome in data])
  B = np.cumsum([outcome[2] for outcome in data])
  Bc = np.cumsum([not outcome[2] for outcome in data])
  ax[1].plot(games, A_B/B, alpha=.7, label="P(win|change)")
  ax[1].plot(games, A_Bc/Bc, alpha=.7, label="P(win|no change)")
  ax[1].plot(games, 2/3*np.ones(games.shape), 'g--', alpha=.5, label="Theoretical P(win|change)")
  ax[1].plot(games, 1/3*np.ones(games.shape), 'r--', alpha=.5, label="Theoretical P(win|no change)")
  # Adjust graphs and commit to screen
  a = list(ax[0].axis())
  a[2] = 0
  a[3] = max(np.max(strategy_counts)+10, 25)
  ax[0].axis(tuple(a))
  a = list(ax[1].axis())
  a[2] = 0
  a[3] = 1
  ax[1].axis(tuple(a))
  ax[1].legend(loc='upper right')
  fig.show()

widgets.interact_manual.opts['manual_name'] = 'Do show!'
widgets.interact_manual(run_game_show, n_runs=widgets.IntSlider(min=1, max=100, step=1, value=1));

interactive(children=(IntSlider(value=1, description='n_runs', min=1), Button(description='Do show!', style=Bu…